### Load Pre-Trained Models

In [75]:
msk_list = [str(i) for i in range(20,60,10)]
noise_list = [str(i) for i in range(0,60,10)]
noise_list

['0', '10', '20', '30', '40', '50']

In [65]:
#import sys  
#sys.path.insert(1, 'src\models')

from models import ssl
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,f1_score,average_precision_score,accuracy_score
import lightning as L
import pandas as pd
import torch
from datamodules import dataset



### Load Data

In [50]:
denoised_base_pth = "..\\data\\sample\\train\\scaled\\denoised\\same_sized\\train_flipped_labels_"
noised_base_pth = "..\\data\\sample\\train\\scaled\\train_flipped_labels_"
test = pd.read_csv("..\\data\\sample\\test\\scaled\\test_scaled.csv")
test_pth = "..\\data\\sample\\test\\scaled\\test_scaled.csv"
denoised_list =[]
noised_list=[]
for i in range(len(noise_list)):
    denoised_list.append(pd.read_csv(denoised_base_pth+noise_list[i]+".csv"))
    noised_list.append(pd.read_csv(noised_base_pth+noise_list[i]+".csv"))
len(denoised_list)


6

In [76]:
pretrain_base_pth = "..\outputs\sample\scaled\pretrain\msk_"
#df = pd.DataFrame(columns=("Masking","Noise","AUC","F1","AP"))

for msk_rate in msk_list:
   
    df_data =[]
    pretrained=ssl.SSLModel.load_from_checkpoint(pretrain_base_pth+msk_rate+".ckpt")
    pretrained.eval()
    
    
    for i in range(len(noised_list)):
        #if i!=0: break
        x_raw = dataset.SSLDataset(denoised_base_pth+noise_list[i]+".csv")
        test_raw = dataset.SSLDataset(test_pth)
        auc=f1=ap=accuracy=0
        cnt=0
       
        train_generator = torch.utils.data.DataLoader(x_raw,batch_size=128)
        test_generator = torch.utils.data.DataLoader(test_raw,batch_size=128)
        emb_total=torch.Tensor()
        lab_total=torch.Tensor()
        for batch in iter(train_generator):
            batch_test = next(iter(test_generator))
            cnt+=1
            with torch.no_grad():
                x = batch[:,:-2]
                labels = batch[:,-1]
                
                x_test = batch_test[:,:-1]
                labels_test = batch_test[:,-1]
                
                emb_per_batch = pretrained(x,0)
                emb_per_batch_test = pretrained(x_test,0)
               
            reg = LogisticRegression().fit(emb_per_batch,labels)
            
            pred = reg.predict(emb_per_batch_test)
            
            
            auc_batch = roc_auc_score(labels_test,pred)
            f1_batch = f1_score(labels_test,pred)
            ap_batch = average_precision_score(labels_test,pred)
            accuracy_batch = accuracy_score(labels_test, pred)
            
            auc+= auc_batch
            f1+= f1_batch
            ap+=ap_batch
            accuracy += accuracy_batch
            
        #auc/=cnt
        #f1/=cnt
        #ap/=cnt
        #accuracy/=cnt
        print("-----"+noise_list[i]+"% noise"+"-----")
        print("AUC: "+str(auc))
        print("F1: "+str(f1))
        print("AP: "+str(ap))
        print("Accuracy: "+str(accuracy))
        #d = {"Masking":msk_rate,"Noise":noise_list[i],}
        df_data.append([msk_rate,noise_list[i],auc,f1,ap,accuracy])
        
    df = pd.DataFrame(df_data,columns=("Masking","Noise","AUC","F1","AP","Accuracy"))
    df.to_csv("C:\\Users\\A200274027\\vscode\\pix\\home\\playground\\ssl_sample\\outputs\\sample\\scaled_denoised_sized\\sft\\"+msk_rate+"_13.csv",index=False,sep=';')




c:\Users\A200274027\vscode\pix\home\playground\ssl_sample\src\datamodules\dataset.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.Tensor(arr)
c:\Users\A200274027\vscode\pix\home\playground\ssl_sample\src\datamodules\dataset.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.Tensor(arr)
c:\Users\A200274027\vscode\pix\home\playground\ssl_sample\src\datamodules\dataset.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use

-----0% noise-----
AUC: 78.62206712206716
F1: 62.463599205305776
AP: 46.11224932907453
Accuracy: 77.5390625


c:\Users\A200274027\vscode\pix\home\playground\ssl_sample\src\datamodules\dataset.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.Tensor(arr)
c:\Users\A200274027\vscode\pix\home\playground\ssl_sample\src\datamodules\dataset.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.Tensor(arr)
c:\Users\A200274027\vscode\pix\home\playground\ssl_sample\src\datamodules\dataset.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use

KeyboardInterrupt: 

-----50% noise-----
AUC: 0.5652650586869599
F1: 0.625811823480433
AP: 0.5358506271504963
